Let's look at this example:

In [1]:
import numpy as np

def fn(a, b):
    return np.multiply.outer(a, b)[5]

a = np.arange(10)
b = np.arange(10)
fn(a, b)

array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45])

What *is* this result?

Well we could say that it is the result to applying the outer product to two arrays and then taking the 5th item. 

However, this way of understanding the operation obscures two key things:

1. We don't need to materialize the whole outer product result if we only use the 10th index of it. 
2. Outer product can be executed as more primitive operations .

For efficiency we don't want to do work we don't have to (1) and since different hardware and software backends have different interfaces, we need to be able to translatate  our operation into equivalent forms (2). 

So what's another way of representing this computation?

Instead of thinking about how to execute the expression as we did above (imperative) let's consider what are the properties of the result (declarative). Because then we can just think about how the result should behave and hopefully that definition will not depend directly on intermediate results, just on the inputs. Then we don't have to compute them and we can rely on more primititve operations.

We know the result is an "array", but what is an array? In NumPy land, an array is expected to behave consistantly under the NumPy API. For example, normal indexing should reduce the dimensionality. But this API is very large! (xxx methtods) So this seems like it would be hard to understand and specify how the result works under all these operations.


---

So let's use the definition from Lenore Mullin's Mathematics of Arrays instead. In her work, which came out the the APL language, arrays have **indexing** and **shape**. Indexing is a function from the indices to an item inside.

So, instead of thinking of outer product as a compositive operation, we can understand how it relates to these concepts. Writing in Python ish syntax:

```python
np.multiply.outer(a, b).shape == a.shape + b.shape
np.multiply.outer(a, b)[*i, *j] == a[i] * b[j]
# where len(i) == len(a.shape)
```

We can also write partial indexing in this form:


```python
a[i].shape == a.shape[1:]
a[i][*j] == a[i, *j]
```

Together we can use these forms to get the index and shape of our final form:


```python
np.multiply.outer(a, b)[10].shape == (a.shape + b.shape)[1:]

# if we know that a and b are both 1D, then:
== ([a.shape[0]] + [b.shape[0]])[1:]
== [a.shape[0], b.shape[0]][1:]
==[b.shape[0]]
# so we know the result is a vector with length the same as the length of b.

# Now let's figure out indexing, by i

np.multiply.outer(a, b)[10][i] == np.multiply.outer(a, b)[10, i]
== a[10] * b[i]
```


So now we have the type of description we were looking for. To restate it:

1. The result has the same shape as `a`
2. Indexing the result with `i` will give you back `a[10] * b[i]`


This description removes the need to compute the intermediary result and it also translates this to more basic operations.

So if we had a backend that only new how to multiply and index, but now how to do outer product (*cough* numba *cough*), then we could use
this mathematical definition of the operation to compile to that backend.

---

We are building `uarray` to give us a place to register these types of translations and make them easily usable.

For example, here we can build up the same operation, but instead of executing it, it builds up an AST:

In [2]:
from uarray import *

In [3]:
A = with_dims(unbound("A"), 1)
B = with_dims(unbound("B"), 1)

op = Index(vector(5), OuterProduct(multiply, A, B))
op

Index(
    Sequence(Int(1), VectorCallable(Scalar(Int(5)))),
    OuterProduct(
        BinaryFunction(
            Scalar(Multiply(Content(Unbound(variable_name="i0")), Content(Unbound(variable_name="i1")))),
            Unbound(variable_name="i0"),
            Unbound(variable_name="i1"),
        ),
        Sequence(
            Length(Unbound(variable_name="A")),
            UnaryFunction(
                Scalar(Content(CallUnary(GetItem(Unbound(variable_name="A")), Unbound(variable_name="i2")))),
                Unbound(variable_name="i2"),
            ),
        ),
        Sequence(
            Length(Unbound(variable_name="B")),
            UnaryFunction(
                Scalar(Content(CallUnary(GetItem(Unbound(variable_name="B")), Unbound(variable_name="i3")))),
                Unbound(variable_name="i3"),
            ),
        ),
    ),
)


Now we can call `replace` to simplify this:

In [22]:
replaced_op = replace(op)
replaced_op

Sequence(
    Length(Unbound(variable_name="B")),
    UnaryFunction(
        Scalar(
            Multiply(
                Content(CallUnary(GetItem(Unbound(variable_name="A")), Int(5))),
                Content(CallUnary(GetItem(Unbound(variable_name="B")), Unbound(variable_name="i42"))),
            )
        ),
        Unbound(variable_name="i42"),
    ),
)


Now we can get the shape and call `replace` which keeps replacing rules until no more match:

In [24]:
print(replace(Shape(replaced_op)))

Sequence(1, <Length(B)>)


We see this is the same as what we had done above.


And now we can index:

In [5]:
i = unbound_with_shape("i", 0)
print(replace(Index(vector_of(i), op)))

Scalar((Content(CallUnary(GetItem(A), 5)) * Content(CallUnary(GetItem(B), Content(i)))))


OK this looks ok, now lets execute it:

In [6]:
A_value = Iota(to_array(10))
B_value = Iota(to_array(10))
subs_op = matchpy.substitute(op, {"A": A_value, "B": B_value})

print("Shape:", replace(Shape(subs_op)))
print("Indexed:", replace(Index(vector_of(i), subs_op)))

Shape: Sequence(1, <10>)
Indexed: Scalar((5 * Content(i)))


Again this looks similar to the manual result above.


---


However, it's still not very tangible.

For that, we can use the `optimize` decorator, which takes a function that takes in NP arrays and returns a new function.

In the middle, we turn it into array expressions as above, then turn those into Python AST:

In [8]:
optimized_fn = optimize(a.shape, b.shape)(fn)

In [12]:
print(optimized_fn.__optimize_steps__["ast_as_source"])



def fn(a, b):
    i_5 = ()
    i_6 = 10
    i_1 = ((i_6,) + i_5)
    i_0 = np.empty(i_1)
    i_2 = 10
    for i_3 in range(i_2):
        i_4 = i_0[i_3]
        i_9 = 5
        i_10 = a
        i_13 = i_10[i_9]
        i_11 = i_3
        i_12 = b
        i_14 = i_12[i_11]
        i_4 = (i_13 * i_14)
        i_0[i_3] = i_4
    return i_0



It has some unneccesary identifiers, but it does what we need at the moment.

We can also look at all the steps produced by the replacement:

In [13]:
all_steps = optimized_fn.__optimize_steps__["all_replaced"]

In [14]:
len(all_steps)

88

We start, just as we did above, with indexing an outer product. However, it is sandwhiched between some operations to turn it into a Python AST:

In [16]:
all_steps[0]

DefineFunction(
    ToNPArray(
        Index(
            Sequence(Int(1), VectorCallable(Scalar(Int(5)))),
            OuterProduct(
                BinaryUfunc(np.ufunc(multiply)),
                Sequence(
                    Int(10),
                    UnaryFunction(
                        Scalar(
                            Content(
                                CallUnary(GetItem(NPArray(Expression(Name("a", Load())))), Unbound(variable_name="i19"))
                            )
                        ),
                        Unbound(variable_name="i19"),
                    ),
                ),
                Sequence(
                    Int(10),
                    UnaryFunction(
                        Scalar(
                            Content(
                                CallUnary(GetItem(NPArray(Expression(Name("b", Load())))), Unbound(variable_name="i20"))
                            )
                        ),
                        Unbound(variable_name="